# Importing

In [6]:
# Basic Packages
import pandas as pd
import numpy as np
import os

# NLP Packages
import nltk 
from nltk.corpus import stopwords

# WordCloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Sklearn Packages
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import ConvergenceWarning
from sklearn.utils import resample
from sklearn.ensemble import BaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Pandas Settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Solve warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Import pickle
import pickle

## Importing DataFrame

In [7]:
df = pd.read_csv('../csv/Hotel_Review_Spell_Checked.csv', index_col=0)

In [46]:
df.head(2)

,Unnamed: 0.1,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Reviews_Clean,Score,Spell_Checked
0,185010,St James Court A Taj Hotel London,No Negative,the location was perfect,9.6,no negative the location was perfect,1,no negative the location was perfect
1,424531,H10 Metropolitan 4 Sup,Nothing,Everything was top notch staff were impeccable,10.0,nothing everything was top notch staff were ...,1,nothing everything was top notch staff were ...


## Importing Lemmatized X and Y Variables

In [8]:
X_lem = pickle.load(open('../pickle/X_lem.pkl', 'rb'))
y_lem = pd.read_pickle('../pickle/y_lem.pkl')

# Preprocessing

In [9]:
stop_words = stopwords.words('english')

In [10]:
df.dropna(inplace=True)

## Train Test Split

In [11]:
X = df.Spell_Checked
y = df.Score

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### Train Test Split with Lemmatization

In [13]:
X_train_lem, X_test_lem, y_train_lem, y_test_lem = train_test_split(X_lem, y_lem, test_size=0.25, random_state=15)

## Count Vectorizer

In [14]:
# Instantiate CountVectorizer
cv = CountVectorizer(stop_words=stop_words)

X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

## TF-IDF

In [15]:
tfidf = TfidfVectorizer(stop_words)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

### TF-DF With Lemmatization

In [16]:
# Save Train and Test data
X_train_lem = tfidf.fit_transform(X_train_lem)
X_test_lem = tfidf.transform(X_test_lem)

## Evaluation Metric

In [17]:
# Evaluation function

def evaluation(y_true, y_pred):
    
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    e_val_dict = {}
    e_val_dict['precision'] = metrics.precision_score(y_true, y_pred)
    print('Evaluation Metrics:')
#     print('Precision: ' + str(e_val_dict))
    print('Accuracy: ' + str(metrics.accuracy_score(y_true, y_pred)))
    print('F1 Score Macro: ' + str(metrics.f1_score(y_true, y_pred, average="macro")))
    print('F1 Score Micro: ' + str(metrics.f1_score(y_true, y_pred, average="micro")))
    print('Recall: ' + str(metrics.recall_score(y_true, y_pred)))
#     return e_val_dict

## Modeling With Count Vectorizer

### Logistic Regression

In [18]:
# Baseline Regression Model
lg_base = LogisticRegression()
lg_base.fit(X_train_cv, y_train) 
y_lg_base_cv = lg_base.predict(X_test_cv)

In [19]:
# Logistic Regression baseline evaluation
evaluation(y_test, y_lg_base_cv)

Evaluation Metrics:
Accuracy: 0.798149861239593
F1 Score Macro: 0.7979942808743291
F1 Score Micro: 0.7981498612395929
Recall: 0.7968582649053909


In [20]:
# # creating dictionary with all metrics
# logreg_precision = precision_score(y_test, y_lg_base_cv)
# logreg_recall = recall_score(y_test, y_lg_base_cv)
# logreg_f1_micro = f1_score(y_test, y_lg_base_cv, average='Micro')
# logreg_f1_micro = f1_score(y_test, y_lg_base_cv, average='Macro')

# metric_dict = {}
# metric_dict['Baseline Logisitic Regression'] = {'precision': logreg_precision, 'recall': logreg_recall, 'f1_score': logreg_f1_score, 'weighted_f1': logreg_f1_weighted}

### Random Forest

In [21]:
rf_cv = RandomForestClassifier(n_jobs=-1)
rf_cv.fit(X_train_cv, y_train)
y_rf_cv = rf_cv.predict(X_test_cv)

In [22]:
# Random Forest baseline evaluation
evaluation(y_test, y_rf_cv)

Evaluation Metrics:
Accuracy: 0.8033302497687327
F1 Score Macro: 0.8033183737472427
F1 Score Micro: 0.8033302497687327
Recall: 0.767583006069261


### Naive Bayes

In [34]:
nb_base_cv = MultinomialNB(alpha = .01)
nb_base_cv.fit(X_train_cv, y_train)

y_nb_base_cv = nb_base_cv.predict(X_test_cv)

In [35]:
# Naive Bayes baseline evaluation
evaluation(y_test, y_nb_base_cv)

Evaluation Metrics:
Accuracy: 0.7811285846438483
F1 Score Macro: 0.7809553525221078
F1 Score Micro: 0.7811285846438483
Recall: 0.7807925740806855


### SVC

In [28]:
svc = SVC(kernel='linear')
svc.fit(X_train_cv, y_train)
y_pred_svc = svc.predict(X_test_cv)

In [29]:
evaluation(y_test, y_pred_svc)

Evaluation Metrics:
Accuracy: 0.7839037927844589
F1 Score Macro: 0.7836872803307368
F1 Score Micro: 0.7839037927844589
Recall: 0.7868618350589075


## Modeling with TF-IDF

### Logistic Regression

In [91]:
# Baseline Regression Model
lg_base_tfidf = LogisticRegression()
lg_base_tfidf.fit(X_train_tfidf, y_train) 
y_lg_base_tfidf = lg_base_tfidf.predict(X_test_tfidf)
# 28 seconds

In [92]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test, y_lg_base_tfidf)

Evaluation Metrics:
Accuracy: 0.8225716928769657
F1 Score Macro: 0.8225714742344841
F1 Score Micro: 0.8225716928769659
Recall: 0.7925740806854695


### Random Forest 

In [93]:
rf_base_tfidf = RandomForestClassifier()
rf_base_tfidf.fit(X_train_tfidf, y_train)
y_rf_base_tfidf = rf_base_tfidf.predict(X_test_tfidf)

In [94]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test, y_rf_base_tfidf)

Evaluation Metrics:
Accuracy: 0.7994449583718779
F1 Score Macro: 0.7993905657529674
F1 Score Micro: 0.7994449583718779
Recall: 0.7554444841128168


### SVC

In [32]:
svc = SVC(kernel='linear')
svc.fit(X_train_tfidf, y_train)
y_pred_svc_tfidf = svc.predict(X_test_tfidf)

In [33]:
evaluation(y_test, y_pred_svc_tfidf)

Evaluation Metrics:
Accuracy: 0.8159111933395005
F1 Score Macro: 0.8159091516682533
F1 Score Micro: 0.8159111933395005
Recall: 0.7840057122456265


In [44]:
svc = SVC(kernel='rbf')
svc.fit(X_train_tfidf, y_train)
y_pred_svc_tfidf = svc.predict(X_test_tfidf)

In [45]:
evaluation(y_test, y_pred_svc_tfidf)

Evaluation Metrics:
Accuracy: 0.8268270120259019
F1 Score Macro: 0.8268019637245034
F1 Score Micro: 0.8268270120259019
Recall: 0.7861478043555873


## TF-IDF With Lemmatization

### Logistic Regression

In [95]:
# Baseline Regression Model
lg_base_tfidf = LogisticRegression(random_state=1)
lg_base_tfidf.fit(X_train_lem, y_train_lem) 
y_lg_base_tfidf_lem = lg_base_tfidf.predict(X_test_lem)
# 28 seconds

In [96]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test_lem, y_lg_base_tfidf_lem)

Evaluation Metrics:
Accuracy: 0.8177613320999075
F1 Score Macro: 0.8177523239045533
F1 Score Micro: 0.8177613320999075
Recall: 0.8019765739385066


### Random Forest

In [97]:
rf_base_tfidf = RandomForestClassifier(random_state=1)
rf_base_tfidf.fit(X_train_lem, y_train_lem) 
y_rf_base_tfidf = rf_base_tfidf.predict(X_test_lem)

In [98]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test_lem, y_rf_base_tfidf)

Evaluation Metrics:
Accuracy: 0.801295097132285
F1 Score Macro: 0.8012336928608115
F1 Score Micro: 0.801295097132285
Recall: 0.7752562225475842


### SVC

In [41]:
svc = SVC(kernel='linear')
svc.fit(X_train_lem, y_train_lem)
y_pred_svc_lem = svc.predict(X_test_lem)

In [43]:
evaluation(y_test_lem, y_pred_svc_lem)

Evaluation Metrics:
Accuracy: 0.8144310823311748
F1 Score Macro: 0.8144111601670185
F1 Score Micro: 0.8144310823311748
Recall: 0.7953879941434846


## SMOTE

### Train Test Split

In [99]:
smote = SMOTE()
X_train_smote_cv, y_train_smote_cv = smote.fit_sample(X_train_cv, y_train) 

smote = SMOTE()
X_train_smote_tfidf, y_train_smote_tfidf = smote.fit_sample(X_train_tfidf, y_train) 

### Logistic Regression - Count Vectorizer

In [100]:
lg_base_smote_cv = LogisticRegression(random_state=1)
lg_base_smote_cv.fit(X_train_smote_cv, y_train_smote_cv)

y_rf_base_smote_cv_smote = lg_base_smote_cv.predict(X_test_cv)

In [101]:
evaluation(y_test, y_rf_base_smote_cv_smote)

Evaluation Metrics:
Accuracy: 0.7994449583718779
F1 Score Macro: 0.7994284740896296
F1 Score Micro: 0.7994449583718779
Recall: 0.7800785433773653


### Random Forest - Count Vectorizer

In [102]:
rf_base_tfidf = RandomForestClassifier(random_state=1)
rf_base_tfidf.fit(X_train_smote_cv, y_train_smote_cv)
y_rf_base_tfidf_smote = rf_base_tfidf.predict(X_test_cv)

In [103]:
evaluation(y_test, y_rf_base_tfidf_smote)

Evaluation Metrics:
Accuracy: 0.8005550416281221
F1 Score Macro: 0.8003552124970327
F1 Score Micro: 0.8005550416281221
Recall: 0.7418779007497323


### Logistic Regression - TF-IDF

In [104]:
# Baseline Regression Model
lg_base_tfidf_smote = LogisticRegression()
lg_base_tfidf_smote.fit(X_train_smote_tfidf, y_train_smote_tfidf) 
y_lg_base_tfidf_smote = lg_base_tfidf_smote.predict(X_test_tfidf)

In [105]:
evaluation(y_test, y_lg_base_tfidf_smote)

Evaluation Metrics:
Accuracy: 0.8227567067530065
F1 Score Macro: 0.822742138540466
F1 Score Micro: 0.8227567067530064
Recall: 0.785076758300607


### Random Forest - TF-IDF

In [106]:
rf_base_tfidf = RandomForestClassifier(random_state=1)
rf_base_tfidf.fit(X_train_smote_tfidf, y_train_smote_tfidf)
y_rf_base_tfidf_smote = rf_base_tfidf.predict(X_test_tfidf)

In [107]:
evaluation(y_test, y_rf_base_tfidf_smote)

Evaluation Metrics:
Accuracy: 0.801295097132285
F1 Score Macro: 0.801167354038615
F1 Score Micro: 0.801295097132285
Recall: 0.7486611924312745


# Pickle Train Test Set

In [118]:
# Pickling Train Set - Features
pickle_out = open("../pickle/X_train_tfidf.pkl",'wb')
pickle.dump(X_train_tfidf, pickle_out)
pickle_out.close()

In [119]:
# Pickling Test Set - Features
pickle_out = open("../pickle/X_test_tfidf.pkl",'wb')
pickle.dump(X_test_tfidf, pickle_out)
pickle_out.close()

In [122]:
# Pickling Train and Test Set = Target
y_train.to_pickle("../pickle/y_train.pkl")
y_test.to_pickle("../pickle/y_test.pkl")

In [ ]:
bag_clf = BaggingClassifier(lg_base_tfidf, n_estimators=500,
                             bootstrap=True, n_jobs=-1, verbose=1)
bag_clf.fit(X_train_tfidf, y_train)

In [ ]:
y_bag_clf = bag_clf.predict(X_test_tfidf)